# Fetch from Openai function

In [7]:
import json
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key="sk-svcacct-8dcUrtOCCiU3jO3ysuZ29D-DeYPWd3Vk_BKZf4s5Ya-UuWekdRLkG2LzQGCKYwIT3BlbkFJ0gsmApPqJnvNjSnyvcQI7UinZ3vehK1O3RJCCCG3qAjxxlUxbFXiQTG2fZwu_AA")

def fetch_random_cities():
    """
    Fetches details about two random cities using the OpenAI API.
    """
    prompt = """
    Provide a JSON object with the following details about two random cities:
    - Name of the city
    - Country of the city
    - Distance between the cities in kilometers
    - Estimated flight time between the cities in hours
    - Fun fact about each city
    - A detailed breakdown of walking and swimming time, assuming:
      - Humans walk 5 km per hour, walking 8 hours per day.
      - Swimming speed is 2 km per hour, swimming 6 hours per day.
      - The walking time is calculated based on an approximate percentage of the total distance being over land, e.g., 70% land and 30% sea. Adjust walking and swimming times accordingly.
    
    Example:
    {
        "city1": {"name": "Paris", "country": "France", "fun_fact": "Paris has a hidden vineyard in Montmartre."},
        "city2": {"name": "New York", "country": "USA", "fun_fact": "New York's Central Park is larger than Monaco."},
        "distance_km": 5837,
        "flight_time_hours": 7.2,
        "walking_swimming_time": "You will need 120 days walking + 60 days swimming."
    }
    """
    try:
        # Create a chat completion request
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ]
        )
        # Extract and parse the response content
        content = response.choices[0].message.content.strip()
        return json.loads(content)
    except json.JSONDecodeError:
        return "Error: Unable to parse the response as JSON."
    except Exception as e:
        return f"An error occurred: {e}"

if __name__ == "__main__":
    cities_info = fetch_random_cities()
    print("City Details:", cities_info)


City Details: {'city1': {'name': 'Tokyo', 'country': 'Japan', 'fun_fact': 'Tokyo is the largest city in the world by population.', 'walking_time_hours': 670, 'swimming_time_hours': 290}, 'city2': {'name': 'Sydney', 'country': 'Australia', 'fun_fact': "Sydney Opera House's shells are made up of over 1 million tiles.", 'walking_time_hours': 530, 'swimming_time_hours': 230}, 'distance_km': 7540, 'flight_time_hours': 10, 'walking_swimming_time': 'You will need 24 days walking + 13 days swimming.'}


# Evaluate guess quality function

In [4]:
def guess_eval(target_distance, user_guess):
    """
    Evaluates the user's guess for the distance between cities.

    Args:
        target_distance (float): The actual distance between the two cities (in kilometers).
        user_guess (float): The user's guessed distance (in kilometers).

    Returns:
        tuple: (is_correct (bool), difference (float))
            - is_correct: True if the guess is exactly correct, False otherwise.
            - difference: The absolute difference between the target distance and the user's guess.
    """
    guess_quality = abs(target_distance - user_guess)
    is_correct = target_distance == user_guess
    return is_correct, guess_quality
    
# Example distances
target_distance = 5837  # Actual distance in kilometers
user_guess = 6000       # User's guessed distance

# Evaluate the guess
result = guess_eval(target_distance, user_guess)
print("Result:", result)  # Output: (False, 163)


Result: (False, 163)


# OPENAI FETCH CITIES FUNCTION ASYNC

In [45]:
import asyncio
import json
from openai import AsyncOpenAI
import nest_asyncio

# Allow nested event loops for interactive environments
nest_asyncio.apply()

# Initialize the OpenAI client
client = AsyncOpenAI(api_key="sk-svcacct-8dcUrtOCCiU3jO3ysuZ29D-DeYPWd3Vk_BKZf4s5Ya-UuWekdRLkG2LzQGCKYwIT3BlbkFJ0gsmApPqJnvNjSnyvcQI7UinZ3vehK1O3RJCCCG3qAjxxlUxbFXiQTG2fZwu_AA")

async def fetch_random_cities():
    prompt = """
    Provide a JSON object with the following details about two random cities:
    - Name of the city
    - Country of the city
    - Distance between the cities in kilometers
    - Estimated flight time between the cities in hours
    - Fun fact about each city
    - A detailed breakdown of walking and swimming time, assuming:
      - Humans walk 5 km per hour, walking 8 hours per day.
      - Swimming speed is 2 km per hour, swimming 6 hours per day.
      - The walking time is calculated based on an approximate percentage of the total distance being over land, e.g., 70% land and 30% sea. Adjust walking and swimming times accordingly.
    
    Example:
    {
        "city1": {"name": "Paris", "country": "France", "fun_fact": "Paris has a hidden vineyard in Montmartre."},
        "city2": {"name": "New York", "country": "USA", "fun_fact": "New York's Central Park is larger than Monaco."},
        "distance_km": 5837,
        "flight_time_hours": 7.2,
        "walking_swimming_time": "You will need 120 days walking + 60 days swimming."
    }
    """

    try:
        # Call the OpenAI API
        response = await client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )

        # Extract and parse the response content
        content = response.choices[0].message.content.strip()

        # Convert the string response to a Python dictionary
        result = json.loads(content)
        return result

    except json.JSONDecodeError:
        return "Error: Unable to parse the response as JSON."
    except Exception as e:
        return f"An error occurred: {e}"

if __name__ == "__main__":
    cities_info = asyncio.run(fetch_random_cities())
    print("City Details:", cities_info)


City Details: {'city1': {'name': 'Tokyo', 'country': 'Japan', 'fun_fact': 'Tokyo is the most populous metropolitan area in the world.', 'walking_time': 125, 'swimming_time': 53}, 'city2': {'name': 'Rio de Janeiro', 'country': 'Brazil', 'fun_fact': 'Rio de Janeiro is home to the iconic Christ the Redeemer statue.', 'walking_time': 87, 'swimming_time': 37}, 'distance_km': 18879, 'flight_time_hours': 23.4, 'walking_swimming_time': 'You will need approximately 212 days walking + 90 days swimming.'}


# Working test code: 

In [5]:
from assets.utils import fetch_random_cities, guess_eval
import streamlit as st


def main():
    st.title("Guess the Distance Between Two Cities")

    if 'game_data' not in st.session_state:
        st.session_state.game_data = fetch_random_cities()
        st.session_state.guesses = []
        st.session_state.rounds = []

    game_data = st.session_state.game_data

    if isinstance(game_data, str):
        st.error(game_data)
        return

    city1 = game_data['city1']
    city2 = game_data['city2']
    target_distance = game_data['distance_km']

    st.write(f"Guess the distance between **{city1['name']}**, {city1['country']} and **{city2['name']}**, {city2['country']}.")

    with st.form(key='guess_form'):
        user_guess = st.number_input("Enter your guess in kilometers:", min_value=0)
        submit_button = st.form_submit_button(label='Submit')

    if submit_button:
        is_correct, difference = guess_eval(target_distance, user_guess)
        st.session_state.guesses.append(difference)

        if is_correct:
            st.success("Congratulations! Your guess is correct.")
            st.session_state.rounds.append({
                'target_distance': target_distance,
                'total_difference': sum(st.session_state.guesses),
                'num_guesses': len(st.session_state.guesses)
            })
            st.session_state.game_data = fetch_random_cities()
            st.session_state.guesses = []
        else:
            st.info(f"Your guess was off by {difference:.2f} km. Try again!")

if __name__ == "__main__":
    main()


2024-11-28 07:52:09.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-28 07:52:09.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar